# Langchain - LCEL
## Introduction

LangChain Expression Language, or LCEL, is a declarative way to easily compose chains together.
It is similar to the Unix pipe methodology to easily combine different smaller code parts and making it easy to replace parts of the *chain*.

See <https://python.langchain.com/v0.1/docs/expression_language/> for more details.

## Installation

In [1]:
%pip install -q langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


## An LCEL chain example with OpenAI



Here's a simple example of creating a chain to generate jokes: it consists of a prompt, model and output_parser. And now that chain can be use on its own.

Example taken from <https://python.langchain.com/v0.1/docs/expression_language/primitives/sequence/>

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

# The chat prompt template
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
# The model
model = ChatOpenAI(model="gpt-4o-mini",temperature=0)

# A parser that just outputs the text
output_parser = StrOutputParser()

# Notice here how we can pipe the prompt to the model and then to the output parser
# similar to unix pipes
chain = prompt | model | output_parser
result = chain.invoke({"topic": "ice cream"})
print(result)

Why did the ice cream truck break down?

Because it had a rocky road!


## Another chain to evaluate the previous chain

Here we build a set of two chains (in Anthropic): one to generate the joke, another to check the joke.

In [3]:
from langchain_anthropic import ChatAnthropic
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

joke_prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = ChatAnthropic(model_name="claude-3-haiku-20240307",temperature=0)

joke_chain = joke_prompt | model | StrOutputParser()
joke_chain.invoke({"topic": "bears"})

from langchain_core.output_parsers import StrOutputParser

analysis_prompt = ChatPromptTemplate.from_template("is this a funny joke? {joke}")

Now we can put it all together in one single chain both generating and judging the joke.

In [4]:
# A chain that both generates the joke and analyses the joke
composed_chain = {"joke": chain} | analysis_prompt | model | StrOutputParser()
composed_chain.invoke({"topic": "bears"})

composed_chain_with_lambda = (
    chain
    | (lambda input: {"joke": input})
    | analysis_prompt
    | model
    | StrOutputParser()
)

analyzed_joke_result = composed_chain_with_lambda.invoke({"topic": "beets"})
print(analyzed_joke_result)

That's a pretty good pun-based joke! The humor comes from the play on words with "root" relationship, which refers to the fact that beets and carrots are both root vegetables.

The joke works because:

1. It takes a common relationship concept (breaking up) and applies it to two vegetables, which is an unexpected and slightly absurd pairing.

2. The pun on "root" relationship is clever and makes the joke more than just a simple vegetable-based joke. It adds an extra layer of wordplay.

3. Puns and plays on words are a common source of humor, as they surprise the listener and make them think about the double meaning.

So in this case, the combination of the unexpected subject matter, the clever pun, and the overall silliness of the premise makes it a fairly amusing and well-constructed vegetable-themed joke. Puns like this can be hit-or-miss, but this one lands pretty well.


The advantage of the LCEL syntax is that is drives resusability of chains similar to Unix commands.